In [1]:
%load_ext jupyter_black

In [1]:
import json
from glob import glob
from pathlib import Path
from typing import NewType, Iterable, Callable

import nvector
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import nvector as nv

# ml
import tensorflow
from sklearn.manifold import LocallyLinearEmbedding

# plotting
import matplotlib.pyplot as plt

# number of seconds in 2 mins
TWO_MINS = 120.0
idx: slice = pd.IndexSlice

FeatureCollection = NewType("FeatureCollection", dict)

wgs84 = nv.FrameE(name="WGS84")

# all_files = sorted(glob("/workspaces/sppp/data/probsevere/2021/**/*.json"))

2022-07-21 14:51:26.371369: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-21 14:51:26.371409: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
from sklearn.manifold import LocallyLinearEmbedding
from shapely.geometry import Polygon

from sppp.transfer.const import GridSpace


grid = GridSpace()
zeros = grid.zeros_grid()


def embed(df: pd.DataFrame, grid: GridSpace) -> pd.DataFrame:

    lle = LocallyLinearEmbedding(n_components=1, n_neighbors=10)
    fresh = df[["PS", "MOTION_EAST", "MOTION_SOUTH"]].copy()
    # cent = df["geometry"].centroid
    fresh["STAB"] = lle.fit_transform(df[["MUCAPE", "MLCAPE", "MLCIN"]])
    # fresh = fresh.astype(np.float32)

    fresh["X"] = np.argmin(abs(df.X.values[:, np.newaxis] - grid.x), axis=1)
    fresh["Y"] = np.argmin(abs(df.Y.values[:, np.newaxis] - grid.y), axis=1)

    return fresh


# fresh = midf.pipe(embed, grid).


def x_any_y(df: pd.DataFrame) -> pd.DataFrame:
    def generator():
        for arr in df["geometry"]:
            point = Polygon(arr.tolist()).centroid
            yield point.x, point.y

    df[["X", "Y"]] = tuple(generator())

    return df


midf = (
    pd.read_parquet("/workspaces/sppp/data/2021-10.parquet")
    .pipe(x_any_y)
    .drop(columns=["geometry"])
    # .pipe(embed, grid)
)

mask = midf.columns[~midf.columns.isin( ("MAXRC_EMISS", "MAXRC_ICECF", "geometry"))]

midf[mask] = midf[mask].astype(np.float32)

midf

MUCAPE  MLCAPE  MLCIN    EBSHEAR  SRH01KM  \
validTime           ID                                                 
2021-10-11 07:36:53 91366  3073.0  2478.0    0.0  56.099998     38.0   
                    92106    99.0     9.0 -999.0  20.700001     46.0   
                    92451  2669.0  2266.0    0.0  49.099998    172.0   
                    92565   652.0   231.0 -128.0  23.500000    198.0   
                    92609   491.0   314.0 -125.0  30.100000    206.0   
...                           ...     ...    ...        ...      ...   
2021-10-11 00:48:54 90328  1931.0  1393.0   -7.0  39.900002     73.0   
                    90329  1318.0   866.0  -21.0  61.099998    165.0   
                    90330  2020.0  1585.0  -16.0  73.900002    200.0   
                    90331  1814.0  1347.0   -1.0  21.799999      0.0   
                    90332  2018.0  1604.0   -4.0  13.100000      4.0   

                           MEANWIND_1-3kmAGL  MESH  VIL_DENSITY  FLASH_RATE  \
validTime           ID                                                        
2021-10-11 07:36:53 91366          11.800000  0.16         0.97         3.0   
                    92106          18.500000  0.00         0.37         0.0   
                    92451          35.099998  0.53         1.45        44.0   
                    92565          40.000000  0.00         0.54         0.0   
                    92609          42.099998  0.00         0.56         0.0   
...                                      ...   ...          ...         ...   
2021-10-11 00:48:54 90328          23.600000  0.00         0.48         0.0   
                    90329          17.799999  0.00         0.40         0.0   
                    90330          25.500000  0.03         0.64         1.0   
                    90331           7.400000  0.00         0.63         0.0   
                    90332           1.600000  0.00         0.92         0.0   

                           FLASH_DENSITY  ...  WETBULB_0C_HGT  PWAT  \
validTime           ID                    ...                         
2021-10-11 07:36:53 91366           0.34  ...            12.9   2.0   
                    92106           0.00  ...             9.2   1.3   
                    92451           0.58  ...            13.1   2.3   
                    92565           0.00  ...            11.3   1.5   
                    92609           0.00  ...            10.3   1.5   
...                                  ...  ...             ...   ...   
2021-10-11 00:48:54 90328           0.00  ...            11.2   1.7   
                    90329           0.00  ...            11.9   1.7   
                    90330           0.03  ...            11.7   1.9   
                    90331           0.00  ...            12.2   1.8   
                    90332           0.00  ...            10.3   1.4   

                           CAPE_M10M30  LJA   SIZE  MOTION_EAST  MOTION_SOUTH  \
validTime           ID                                                          
2021-10-11 07:36:53 91366        671.0  0.0  244.0        8.415         1.715   
                    92106          0.0  0.0  812.0       -0.573        -2.552   
                    92451        689.0  0.1  283.0       10.628        -1.365   
                    92565        173.0  0.0   84.0        5.947       -10.048   
                    92609        119.0  0.0  816.0        3.173        -7.111   
...                                ...  ...    ...          ...           ...   
2021-10-11 00:48:54 90328        545.0  0.0   55.0       12.263        -9.197   
                    90329        295.0  0.0   72.0        7.236        -9.496   
                    90330        631.0  0.0   80.0       14.995        -9.057   
                    90331        378.0  0.0   43.0       -2.758        -1.557   
                    90332        506.0  0.0   74.0       -4.353        -4.430   

                             PS          X          Y  
validTime           ID      

In [10]:
midf["X"].values[:, np.newaxis] - grid.x
# midf[["X", "Y"]]
# np.argmin(abs(midf.X.values[:, np.newaxis] - grid.x), axis=1)

: 

: 

In [ ]:
# data transformation and initial loading
# def open_file(filepath: Path) -> FeatureCollection:
#     with filepath.open("rb") as fin:
#         return json.load(fin)


# def to_dataframe(fc: FeatureCollection) -> pd.DataFrame:
#     df = gpd.GeoDataFrame.from_features(fc["features"])
#     df["validTime"] = pd.to_datetime(fc["validTime"], format="%Y%m%d_%H%M%S %Z")
#     df["CENTROID"] = df["geometry"].centroid

#     def ecef_vector():
#         for point in df["geometry"].centroid:
#             geo_point = wgs84.GeoPoint(
#                 longitude=point.x, latitude=point.y, degrees=True
#             )
#             yield geo_point.to_ecef_vector()

#     df["ECEF_VECTOR"] = tuple(ecef_vector())

#     df = df.set_index(["validTime", "ID"])
#     return df
#     # df =


# def to_midf() -> pd.DataFrame:
#     def generate():
#         for file in all_files:
#             fc = open_file(Path(file))
#             yield to_dataframe(fc)

#     return pd.concat(generate())


# midf = to_midf()
# midf

In [ ]:
# helper function
from sklearn.manifold import LocallyLinearEmbedding
from sppp.transfer.const import GridSpace


grid = GridSpace()


def embed(df: pd.DataFrame, grid: GridSpace) -> pd.DataFrame:

    lle = LocallyLinearEmbedding(n_components=1, n_neighbors=10)
    fresh = df[["PS", "MOTION_EAST", "MOTION_SOUTH"]].copy()
    cent = df["geometry"].centroid
    fresh["STAB"] = lle.fit_transform(midf[["MUCAPE", "MLCAPE", "MLCIN"]])
    fresh = fresh.astype(np.float32)

    fresh["X"] = np.argmin(abs(cent.x.values[:, np.newaxis] - grid.x), axis=1)
    fresh["Y"] = np.argmin(abs(cent.y.values[:, np.newaxis] - grid.y), axis=1)

    return fresh


zeros = grid.zeros_grid()


fresh = midf.pipe(embed, grid).loc[idx[:, "89519"], :]

In [ ]:
# machine learning support classes
import keras

# from tensorflow import keras
from keras.engine.sequential import Sequential

# keras:K


class State:
    __has_state: bool = False

    def __init__(self):
        self.__latests: pd.DataFrame = None
        self.__state: pd.DataFrame = None

    def __repr__(self) -> str:
        return self.__state.__repr__()

    @property
    def latests(self) -> pd.DataFrame:
        return self.__latests

    def set_storm(self, df: pd.DataFrame) -> None:

        self.__latests = df

        if not self.__has_state:
            self.__has_state = True
            self.__state = df
        else:
            old = self.__state
            self.__state = pd.concat([old, df])

    def iterstorms(self):
        df = self.__state.iloc[-2:].groupby("ID")
        for id, x in self.__state.iloc[-2:].groupby("ID"):
            yield x

    def has_hist(self) -> bool:
        return isinstance(self.__state, pd.DataFrame)

    @property
    def frame(self) -> pd.DataFrame:
        return self.__state


def build_model(frame_b: pd.DataFrame) -> Sequential:
    model = keras.models.Sequential(
        [
            keras.layers.Dense(
                22.5,
                activation="elu",
                input_shape=frame_b.shape,
            ),
            keras.layers.Dense(
                22.5,
                activation="sigmoid",
            ),
            keras.layers.Dense(3),
        ]
    )
    return model


def make_reward_map() -> np.ndarray:
    """creates 11x11 matrix"""
    a = list(np.linspace(0, 0.25, 5))
    a2 = np.array(a + [0.5] + list(reversed(a)))
    return a2 + a2[:, np.newaxis]

In [ ]:
import tensorflow as tf
import gym

from sppp.transfer.funcs import mask_frames_by_id
from sppp.transfer.const import GridSpace

grid = GridSpace()


class SPPPEnv(gym.Env):
    def __init__(self, state: "State") -> None:
        self.state = state

    def _compute_reward(self) -> int:
        return 1

    def step(self, action: np.ndarray):
        # action is produced by DQN, action is discrete
        # self.cache.move(action)
        # compute reward based on state(position) of the car
        # storm_state = self.car_agent.getCarState()
        # reward = self._compute_reward(storm_state)
        # # check if the episode is done
        # car_controls = self.car_agent.getCarControls()
        # done = self._isDone(storm_state, car_controls, reward)
        # # log info
        # info = {}
        # # observation is RGB image from car's camera
        # observation = self.car_agent.observe()
        observation = self.state.latests
        reward = self._compute_reward()
        done = False
        info = {}
        return observation, reward, done, info

    @property
    def observation_space(self) -> pd.DataFrame:
        return self.state.latests


def iterframe(df: pd.DataFrame) -> Iterable[tuple[pd.Timestamp, pd.DataFrame]]:
    yield from df.groupby("validTime")


n_inputs = 4
state = State()
env = SPPPEnv(state)
loss = keras.losses.binary_crossentropy
reward_map = make_reward_map()


n_outputs = 5
box_shape = 5


def reward_matrix(x: int, y: int, box_size: int = 5):
    gamefield = (
        zeros.loc[
            y - box_size : y + box_size,
            x - box_size : x + box_size,
        ].copy()
        + reward_map
    )
    return gamefield.values


transition_probs = [[0, 0, 1] * 20] * 101
possible_actions = [[0, 1, 2], [0, 2], [1]] * 101  # [list(range(-5, 5))] * 101


def swp_class_ploicy(
    state: int,  # probiblity of severe weather
    epsilon=0,
):
    if np.random.rand() < epsilon:
        return np.random.randint()
    # v = model.predict(state[np.newaxis])
    return np.random.choice(possible_actions[state])


def training_step(
    env: SPPPEnv,
    obs: np.ndarray,
    model: Sequential,
    loss_fn: Callable[[any], any],
):
    """policy gradient"""
    # with tf.GradientTape() as tape:
    #     left_prob = model(obs[np.newaxis, :])
    #     action = tf.random.uniform([1, 1]) > left_prob

    #     y_target = tf.constant([[1.0]]) - tf.cast(action, tf.float32)

    #     loss = tf.reduce_mean(loss_fn(y_target, left_prob))

    # grads = tape.gradient(loss, model.trainable_variables)
    # return action.numpy()


class SevereWeatherClassifcation:
    def policy(self, series: pd.Series) -> np.ndarray:
        return 1


swc = SevereWeatherClassifcation()

if __name__ == "__main__":
    for vt, df in iterframe(fresh):
        track_rewards = 0
        obs = env.reset()
        # evaluate prediction
        if state.has_hist():
            # there is existing storm information normalize frame_a and frame_b by the the in's in the index
            frame_a, frame_b = mask_frames_by_id(state.latests, df)
            # assert that the frames are of an equal shape
            assert frame_a.shape == frame_b.shape
            bg = frame_b[["MOTION_EAST", "MOTION_SOUTH", "STAB"]].values

            # x = play_one_step(env, bg, model, loss)
            # obs, reward, done, info = env.step(frame_b[["MOTION_EAST", "MOTION_SOUTH"]])
            with tf.GradientTape() as tape:
                for x, s in frame_b.iterrows():
                    model = build_model(s)
                    reward = grid.reward_matrix(s.X, s.Y)
                    action = swc.policy(s)
                    # action = swp_classification_policy(int(s["PS"]))
                    # reward = reward_matrix(s.X, s.Y)
                # ...
            # for tup in frame_b.itertuples():
            #         model.predict()
            # training_step(env, frame_b[["X", "Y"]].values, model, loss)
            # prob = step(0, action)

        state.set_storm(df)
# prob
# frame_b
reward
# np.random.choice(a=[1], p=prob)
# gamefield

In [ ]:
reward.stack().plot()